In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

/home/peta/petaLab/bayesian-iroha/.venv/lib/python3.11/site-packages/arviz/__init__.py:39: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


In [4]:
print(f"PyMC version: {pm.__version__}")

# 再現性の確保
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

PyMC version: 5.26.1


In [102]:
# --- 1. Data の生成 (N=20 小規模Data) ---
def generate_weighted_scenario_data(n_per_page=20):
    """
    Top Page 改善、Checkout page 悪化の Trap-data 生成
    """
    pages = ["Top", "Detail", "Contract", "Checkout"]

    scenario = {
        'Top': (3000, 2500),  # 改善（良）
        'Detail': (2800, 2600),  # （微良）
        'Contract': (3500, 3550),  # 変化なし(微増)
        'Checkout': (3000, 4000)  #  悪化　★ここが罠
    }

    data = []
    for page in pages:
        mu_pre, mu_post = scenario[page]

        # Pre (対数正規分布)
        lcp_pre = rng.lognormal(mean=np.log(mu_pre), sigma=0.4, size=n_per_page)
        data.extend([{"page": page, "group": "pre", "lcp": val} for val in lcp_pre])

        # Post (対数正規分布)
        lcp_post = rng.lognormal(mean=np.log(mu_post), sigma=0.4, size=n_per_page)
        data.extend([{"page": page, "group": "Post", "lcp": val} for val in lcp_post])

    return pd.DataFrame(data)

In [103]:
df = generate_weighted_scenario_data(n_per_page=20)

In [104]:
df.groupby("group").agg({"lcp": "mean"})

,lcp
group,
Post,3368.540709
pre,3367.991907


単純に `df.groupby("group").agg({"lcp": "mean"})` を見ると、Sample数が均等なため
- Pre: 3368ms -> Post: 3369ms

と、全体として大きな変化はありません。

In [106]:
df.groupby(["page", "group"]).agg({"lcp": "mean"}).unstack()  # 確認

# unstack() は、MultiIndex（重層的なインデックス）を持つ Series や DataFrame のインデックスの最下層を、カラム（列）へと展開（ピボット）する効果があります。
# このコードの場合、groupby(["page", "group"]) によってインデックスが page と group の二段構えになりますが、
# .unstack() を付けることで、group（pre/Post）が横に並び、ページごとの比較がしやすくなります。

lcp             
group            Post          pre
page                              
Checkout  4526.429897  3039.637570
Contract  3798.844753  3854.428841
Detail    2631.132025  3461.750649
Top       2517.756163  3116.150570

しかし、Page毎の集計を確認すると`checkout` が明確に悪化している。
この「局所的な悪化」を Bayesian　Model で確実に捉えていきます。